
# ATSC PBIL: Pipeline Notebook

Notebook này chạy tuần tự các bước:
1) **Build graph** từ dữ liệu SUMO → `graph.json`, `tls_candidates.json`  
2) **Chạy baselines** (No-ATSC / All-ATSC)  
3) **(Tuỳ chọn)** Chạy **PBIL**  
4) **Tổng hợp & trực quan hoá** kết quả

> Gợi ý: đặt file này vào thư mục `scripts/` của repo: `scripts/atsc_pipeline.ipynb`.



## 0) Chuẩn bị môi trường

- Bảo đảm bạn có Python env với các dependency trong `pyproject.toml` đã cài đặt.
- Đặt biến môi trường `SUMO_HOME` (và SUMO đã trong PATH nếu cần).

In [1]:

# Kiểm tra phiên bản Python & vị trí python
import sys, os, subprocess, json, pathlib, platform
print("Python:", sys.version)
print("Executable:", sys.executable)
print("CWD:", os.getcwd())
print("Platform:", platform.platform())
print("SUMO_HOME:", os.environ.get("SUMO_HOME", "(chưa đặt)"))

# (Tuỳ chọn) In ra các gói đã cài
# !pip list | head -n 50


Python: 3.12.4 (tags/v3.12.4:8e8a4ba, Jun  6 2024, 19:30:16) [MSC v.1940 64 bit (AMD64)]
Executable: c:\Users\NghiaNL\AppData\Local\pypoetry\Cache\virtualenvs\choose-atsc-pbil-GIRhaIaW-py3.12\Scripts\python.exe
CWD: e:\VTS\ITS\Choose ATSC\Software\choose_ATSC_PBIL\scripts
Platform: Windows-11-10.0.26100-SP0
SUMO_HOME: C:\Program Files (x86)\Eclipse\Sumo\



## 1) Tham số pipeline

Chỉnh sửa các biến dưới đây cho phù hợp cấu trúc repo của bạn.


In [5]:

from pathlib import Path
import datetime, json, os, sys

# === Paths tương đối so với root repo ===
# REPO_ROOT = Path.cwd()  # nếu notebook đặt tại scripts/ thì REPO_ROOT.parent.parent... điều chỉnh nếu cần

# Nếu bạn đặt notebook ở scripts/, thay bằng: REPO_ROOT = Path.cwd().parent
REPO_ROOT = Path.cwd().parent

DATA_INPUT_SUMO = REPO_ROOT / "data" / "input" / "sumo"
DATA_PROCESSED  = REPO_ROOT / "data" / "processed"
DATA_RESULTS    = REPO_ROOT / "data" / "results"
CONFIGS_DIR     = REPO_ROOT / "configs"

# File cấu hình ví dụ cho PBIL / logging
EXPERIMENT_CFG  = CONFIGS_DIR / "experiment.example.json"
LOGGING_YAML    = CONFIGS_DIR / "logging.yaml"

# Thời gian chạy đặt tên thư mục kết quả nếu dùng thủ công
RUN_STAMP = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

for p in [DATA_INPUT_SUMO, DATA_PROCESSED, DATA_RESULTS, CONFIGS_DIR]:
    print(p, "OK" if p.exists() else "MISSING")


e:\VTS\ITS\Choose ATSC\Software\choose_ATSC_PBIL\data\input\sumo OK
e:\VTS\ITS\Choose ATSC\Software\choose_ATSC_PBIL\data\processed OK
e:\VTS\ITS\Choose ATSC\Software\choose_ATSC_PBIL\data\results OK
e:\VTS\ITS\Choose ATSC\Software\choose_ATSC_PBIL\configs OK



## 2) Build graph (`src/cli/build_graph.py`)

Sinh `graph.json` và `tls_candidates.json` từ dữ liệu SUMO.


In [10]:

import sys, subprocess, os
from pathlib import Path

cmd = [
    sys.executable, "-m", "choose_atsc_pbil.cli.run_baselines"
]

print(" ".join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout)
if res.returncode != 0:
    print(res.stderr)
    raise SystemExit(res.returncode)

print("Build graph hoàn tất.")


c:\Users\NghiaNL\AppData\Local\pypoetry\Cache\virtualenvs\choose-atsc-pbil-GIRhaIaW-py3.12\Scripts\python.exe -m choose_atsc_pbil.cli.run_baselines

Build graph hoàn tất.



## 3) Chạy baselines (`src/cli/run_baselines.py`)

Sinh `baseline_metrics.json` (No-ATSC / All-ATSC).


In [ ]:

import sys, subprocess

cmd = [
    sys.executable, "-m", "src.cli.run_baselines",
    "--graph", str(DATA_PROCESSED / "graph.json"),
    "--tls", str(DATA_PROCESSED / "tls_candidates.json"),
    "--out", str(DATA_PROCESSED / "baseline_metrics.json"),
    "--logging", str(LOGGING_YAML),
    # Tham số ví dụ thêm (bật/tắt warmup, seed, v.v.). Điều chỉnh tùy theo argparse của bạn:
    # "--warmup", "300",
    # "--seed", "42",
]
print(" ".join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout)
if res.returncode != 0:
    print(res.stderr)
    raise SystemExit(res.returncode)

print("Baselines hoàn tất.")



## 4) (Tuỳ chọn) Chạy PBIL (`src/cli/run_pbil.py`)

Tạo thư mục `data/results/runs/<timestamp>/` chứa `best_overall.json`, `run_config_snapshot.json`, v.v.


In [ ]:

import sys, subprocess

# Bật/tắt theo nhu cầu
RUN_PBIL = True

if RUN_PBIL:
    cmd = [
        sys.executable, "-m", "src.cli.run_pbil",
        "--config", str(EXPERIMENT_CFG),
        "--logging", str(LOGGING_YAML),
        # Ví dụ override thư mục kết quả (nếu script hỗ trợ):
        # "--outdir", str(DATA_RESULTS / "runs" / RUN_STAMP),
    ]
    print(" ".join(cmd))
    res = subprocess.run(cmd, capture_output=True, text=True)
    print(res.stdout)
    if res.returncode != 0:
        print(res.stderr)
        raise SystemExit(res.returncode)

    print("PBIL hoàn tất.")
else:
    print("Bỏ qua bước PBIL (RUN_PBIL = False).")



## 5) Tổng hợp & tìm run mới nhất


In [ ]:

from pathlib import Path
import json

def latest_run_dir(results_root: Path) -> Path | None:
    runs = sorted((results_root / "runs").glob("*"))
    if not runs:
        return None
    return runs[-1]

latest = latest_run_dir(DATA_RESULTS)
print("Latest run:", latest)

best_overall = None
if latest and (latest / "best_overall.json").exists():
    with open(latest / "best_overall.json", "r", encoding="utf-8") as f:
        best_overall = json.load(f)

baseline = None
if (DATA_PROCESSED / "baseline_metrics.json").exists():
    with open(DATA_PROCESSED / "baseline_metrics.json", "r", encoding="utf-8") as f:
        baseline = json.load(f)

print("Có best_overall.json:", best_overall is not None)
print("Có baseline_metrics.json:", baseline is not None)



## 6) Trực quan hoá

### 6.1) So sánh Baseline (No-ATSC vs All-ATSC)


In [ ]:

import json
import matplotlib.pyplot as plt
from pathlib import Path

# Kỳ vọng baseline_metrics.json dạng:
# {
#   "No-ATSC": {"avg_delay": ..., "throughput": ..., "queue_len": ...},
#   "All-ATSC": {"avg_delay": ..., "throughput": ..., "queue_len": ...}
# }

baseline_path = DATA_PROCESSED / "baseline_metrics.json"
if baseline_path.exists():
    with open(baseline_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    metrics = sorted(next(iter(data.values())).keys())
    methods = list(data.keys())

    for metric in metrics:
        plt.figure()
        vals = [data[m][metric] for m in methods]
        plt.bar(methods, vals)
        plt.title(f"Baseline comparison: {metric}")
        plt.xlabel("Method")
        plt.ylabel(metric)
        plt.show()
else:
    print("Không tìm thấy baseline_metrics.json để vẽ.")



### 6.2) PBIL so với Baseline


In [ ]:

import matplotlib.pyplot as plt
import json

# Kỳ vọng best_overall.json tối thiểu có:
# {
#   "score": {"avg_delay": ..., "throughput": ..., ...},
#   "config": {...},
#   "meta": {...}
# }

if best_overall and baseline:
    metrics = sorted(best_overall["score"].keys())
    for metric in metrics:
        plt.figure()
        xs, ys = [], []
        # PBIL
        xs.append("PBIL-best")
        ys.append(best_overall["score"].get(metric, None))
        # Baselines
        for name, vals in baseline.items():
            xs.append(name)
            ys.append(vals.get(metric, None))
        plt.bar(xs, ys)
        plt.title(f"PBIL vs Baselines: {metric}")
        plt.xlabel("Method")
        plt.ylabel(metric)
        plt.show()
else:
    print("Thiếu dữ liệu để so sánh PBIL và baseline.")



### 6.3) Inspect cấu hình tốt nhất (best_overall)


In [ ]:

import pprint, json

if best_overall:
    print("=== SCORE ===")
    pprint.pprint(best_overall.get("score", {}), compact=True)
    print("\n=== META ===")
    pprint.pprint(best_overall.get("meta", {}), compact=True)
    print("\n=== CONFIG (rút gọn) ===")
    cfg = best_overall.get("config", {})
    keys = list(cfg.keys())
    preview = {k: cfg[k] for k in keys[:20]}  # in rút gọn
    pprint.pprint(preview, compact=True)
else:
    print("Chưa có best_overall.json")



## Phụ lục: Chạy các script trực tiếp

Nếu bạn muốn gọi trực tiếp `sim_runner.py` hoặc controller tuỳ chỉnh, có thể thêm cell tương tự sau và điều chỉnh tham số.


In [ ]:

# Ví dụ minh hoạ — điều chỉnh theo argparse thực tế của bạn:
# import sys, subprocess
# cmd = [sys.executable, "-m", "src.sim.sim_runner", "--config", str(EXPERIMENT_CFG)]
# print(" ".join(cmd)); res = subprocess.run(cmd, text=True); print(res.returncode)
